In [0]:
import base64
import os
import numpy as np
import pandas as pd
import requests

# This function submits an assignment.  You can submit an assignment as much as you like, only the final
# submission counts.  The paramaters are as follows:
# data - Pandas dataframe output.
# key - Your student key that was emailed to you.
# no - The assignment class number, should be 1 through 1.
# source_file - The full path to your Python or IPYNB file.  This must have "_class1" as part of its name.  
# .             The number must match your assignment number.  For example "_class2" for class assignment #2.
def submit(data,key,no,source_file=None):
    if source_file is None and '__file__' not in globals(): raise Exception('Must specify a filename when a Jupyter notebook.')
    if source_file is None: source_file = __file__
    suffix = '_class{}'.format(no)
    if suffix not in source_file: raise Exception('{} must be part of the filename.'.format(suffix))
    with open(source_file, "rb") as image_file:
        encoded_python = base64.b64encode(image_file.read()).decode('ascii')
    ext = os.path.splitext(source_file)[-1].lower()
    if ext not in ['.ipynb','.py']: raise Exception("Source file is {} must be .py or .ipynb".format(ext))
    r = requests.post("https://api.heatonresearch.com/assignment-submit",
        headers={'x-api-key':key}, json={'csv':base64.b64encode(data.to_csv(index=False).encode('ascii')).decode("ascii"),
        'assignment': no, 'ext':ext, 'py':encoded_python})
    if r.status_code == 200:
        print("Success: {}".format(r.text))
    else: print("Failure: {}".format(r.text))

In [0]:

import os
import pandas as pd
from scipy.stats import zscore
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
import pandas as pd
import io
import requests
import numpy as np
from sklearn import metrics
from sklearn.model_selection import KFold
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from scipy.stats import zscore

# This is your student key that I emailed to you at the beginnning of the semester.
key = "zxSDPsLAb39xNBkgUJ9n58F5bhE5cIlG90IJzIhI"  # This is an example key and will not work.

# You must also identify your source file.  (modify for your local setup)
# file='/resources/t81_558_deep_learning/assignment_yourname_class1.ipynb'  # IBM Data Science Workbench
# file='C:\\Users\\jeffh\\projects\\t81_558_deep_learning\\t81_558_class1_intro_python.ipynb'  # Windows
#file='/Users/jheaton/projects/t81_558_deep_learning/assignments/assignment_yourname_class5.ipynb'  # Mac/Linux
# file = "C:\\Users\\jeffh\\Dropbox\\school\\teaching\\wustl\\classes\\T81_558_deep_learning\\solutions\\assignment_solution_class5.ipynb"

# Begin assignment
df = pd.read_csv("reg-33-data.csv")

# Encode the feature vector
ids = df['id']
df.drop('id',1,inplace=True)

In [0]:
df.head(5)

In [0]:
df.columns

In [0]:
numerical_cols = ['height','max','number','length','power','weight']
text_cols = ['convention','cat2','usage','region','code','item','country']

In [0]:
#fill nan with mean vlaue
for column in list(df.columns[df.isnull().sum() > 0]):
    mean_val = df[column].mean()
    df[column].fillna(mean_val, inplace=True)
df.isnull().sum()>0

In [0]:
#normalize
for col in numerical_cols:
    df.loc[:,col] = zscore(df[col].values)

In [0]:
for col in text_cols:
    #print(col)
    df = df.join(pd.get_dummies(df[col],prefix=col))
new_df = df.drop(columns=text_cols)

In [0]:
new_df.head(5)

In [0]:
train = new_df[[x for x in list(new_df.columns) if x != 'target']].values
test = new_df['target'].values

print(train.shape, test.shape)

In [0]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import Adadelta
from tensorflow.keras.optimizers import SGD

def model():
    model=Sequential()
    model.add(Dense(1000, activation='relu', input_dim=256))
    model.add(Dense(1000, activation='relu'))
    model.add(Dense(512, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(1))
    adam = Adam(learning_rate=0.005, beta_1=0.9, beta_2=0.999, amsgrad=True)
    ada = Adadelta(learning_rate=0.002, rho=0.9)
    sgd = SGD(lr=0.002, decay=1e-3, momentum=0.9, nesterov=True)
    model.compile(loss='mse', optimizer=adam)
    return model

In [0]:
from sklearn.model_selection import KFold
import numpy as np

model1 = model()
    

kf = KFold(n_splits=5)
kf.get_n_splits(train)
cnt = 1

for train_index, test_index in kf.split(train):
    X_train, X_test = train[train_index], train[test_index]
    y_train, y_test = test[train_index], test[test_index]
    if cnt == 1:
        model1.fit(X_train, y_train, batch_size=128, epochs=100, verbose=0)
        model1.evaluate(X_test, y_test, verbose=1)
        cnt += 1
    elif cnt == 2:
        model1.fit(X_train, y_train, batch_size=128, epochs=100, verbose=0)
        model1.evaluate(X_test, y_test, verbose=1)
        cnt += 1
    elif cnt == 3:
        model1.fit(X_train, y_train, batch_size=128, epochs=100, verbose=0)
        model1.evaluate(X_test, y_test, verbose=1)
        cnt += 1
    elif cnt == 4:
        model1.fit(X_train, y_train, batch_size=128, epochs=100, verbose=0)
        model1.evaluate(X_test, y_test, verbose=1)
        cnt += 1
    else:
        model1.fit(X_train, y_train, batch_size=128, epochs=100, verbose=0)
        model1.evaluate(X_test, y_test, verbose=1)
        break

In [0]:
predictions = model2.predict(train)
np.sqrt(((predictions - test) ** 2).mean())

In [0]:
model2 = model()
model2.fit(train, test, batch_size=1024, epochs=1000, verbose=1)

In [0]:
new_df = ids.to_frame()
new_df['y'] = df['target']
new_df['pred'] = pd.Series(predictions.reshape(-1))

In [0]:
new_df['diff'] = new_df['y'] - new_df['pred']
new_df['diff'] = new_df['diff'].apply(lambda x: abs(x))

In [0]:
new_df.head(5)